# Импорты и переменные

In [1]:
!pip install tqdm
!pip install torch
!pip install seqeval #для
#!pip install -q deeppavlov # модель для задачи 2
#!python -m deeppavlov install ner_ontonotes_bert #для задачи 2
!pip install corus # для ленты новостей
!pip install pymorphy3 #для ленты новостей
!pip install datasets
!pip install evaluate # замер метрик


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 77.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [2]:
#!wget https://github.com/yutkin/Lenta.Ru-News-Dataset/releases/download/v1.0/lenta-ru-news.csv.gz # лента. датасет для задачи 2

!wget http://www.labinform.ru/pub/named_entities/collection5.zip #датасет NER
!unzip collection5.zip #датасет NER
!rm collection5.zip #датасет NER
path = '/content/Collection5' #датасет NER

--2025-04-13 20:40:16--  http://www.labinform.ru/pub/named_entities/collection5.zip
Resolving www.labinform.ru (www.labinform.ru)... 95.181.230.181
Connecting to www.labinform.ru (www.labinform.ru)|95.181.230.181|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1899530 (1.8M) [application/zip]
Saving to: ‘collection5.zip’

collection5.zip     100%[===================>]   1.81M  1.11MB/s    in 1.6s    

2025-04-13 20:40:19 (1.11 MB/s) - ‘collection5.zip’ saved [1899530/1899530]

Archive:  collection5.zip
   creating: Collection5/
  inflating: Collection5/001.ann     
  inflating: Collection5/001.txt     
  inflating: Collection5/002.ann     
  inflating: Collection5/002.txt     
  inflating: Collection5/003.ann     
  inflating: Collection5/003.txt     
  inflating: Collection5/004.ann     
  inflating: Collection5/004.txt     
  inflating: Collection5/005.ann     
  inflating: Collection5/005.txt     
  inflating: Collection5/006.ann     
  inflating: Collection

In [3]:
#import numpy as np
import pandas as pd

import zipfile
import os
from corus import load_ne5
from pathlib import Path

from tqdm import tqdm
from corus import load_lenta
from datasets import Dataset
import evaluate
from transformers import TrainingArguments, Trainer, DataCollatorForTokenClassification
import os
from transformers import AutoTokenizer, AutoModelForTokenClassification

os.environ['WANDB_DISABLED'] = 'true' # отключаем логирование по api для berta


# Загружаем стоп-слова
#nltk.download('stopwords')
#stop_words = set(stopwords.words('russian'))

# Загружаем данные
#path = 'lenta-ru-news.csv.gz'
#records = load_lenta(path)

#max_samples = 20000 # обрежем датасет

# Загрузка, получение меток, подготовка в формат для обучения

In [4]:
path = Path('/content/Collection5')
records = list(load_ne5(path)) #загрузили. теперь один record — это один размеченный текст

print(records[0]) #чекаем пример записи

Ne5Markup(id='233', text='Президент России уволил 6 генералов МВД\r\n \r\nПрезидент России Дмитрий Медведев продолжает перестановки в руководстве МВД. Сегодня он уволил 6 генералов милиции.\r\n\r\nВ частности, указом президента отправлен в отставку начальник Центра обеспечения оперативно-служебной деятельности по противодействию экстремизму МВД генерал-майор милиции Владимир Булатов. Не взяли на работу в полицию и ряд региональных руководителей ведомства. Уволены начальник Управления на транспорте МВД по Северо-Кавказскому федеральному округу генерал-майор милиции Игорь Жуков, начальник Управления внутренних дел по Тверской области генерал-майор милиции Александр Куликов, начальник Управления внутренних дел по Тамбовской области генерал-майор милиции Владимир Фурсов.\r\n\r\nТакже президент отправил в отставку начальников 2 милицейских институтов: начальника Сибирского юридического института МВД РФ генерал-майора милиции Николая Михайлова и начальника Тюменского юридического института М

In [5]:
def record_to_dict_bio(record):
    '''
    Функция преобразует один record в формат, пригодный для обучения модели NER — список токенов и BIO-теги (B-начало, I-продолжение, O-вне сущности).
    только размеченные сущности.
    '''
    tokens = []
    tags = []

    for span in record.spans: #разбиваем текст сущности на отдельные слова
        words = span.text.split()
        if not words:
            continue

        for i, word in enumerate(words):
            tokens.append(word)
            if i == 0: # Для первого слова ставим метку B-TYPE
                tags.append(f'B-{span.type}')
            else: # для последующих — I-TYPE
                tags.append(f'I-{span.type}')

    return {'tokens': tokens, 'ner_tags': tags}

data = [record_to_dict_bio(r) for r in records]

In [6]:
all_tags = sorted(set(tag for item in data for tag in item['ner_tags'])) #отсортированный список уникальных тегов
tag2id = {tag: i for i, tag in enumerate(all_tags)}
id2tag = {i: tag for tag, i in tag2id.items()}

# Преобразуем теги в id
for item in data:
    item['ner_tags'] = [tag2id[tag] for tag in item['ner_tags']]

dataset = Dataset.from_list(data)

# Раделение train/test

In [7]:
dataset = dataset.train_test_split(test_size=0.2, seed=333)

# Загрузка модели и токенизатора


In [8]:
model_name = 'cointegrated/rubert-tiny2' # русскоязычная BERT модель, уменьшенная, подходит для задача NLP и уже обучена в том числе NER меткам
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(
    model_name,
    num_labels=len(tag2id),
    id2label=id2tag,
    label2id=tag2id
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/401 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.74M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/118M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
def tokenize_and_align_labels(examples):
    # преобразуем слова в индексы по позициям в словаре
    tokenized_inputs = tokenizer(
        examples['tokens'], #список токенов для каждого списка слов
        truncation=True,
        is_split_into_words=True
    )
    # подготовка меток ner_tags
    labels = []
    for i, label in enumerate(examples['ner_tags']):
        word_ids = tokenized_inputs.word_ids(batch_index=i) #для каждого токена мы получаем его индекс в исходном тексте
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids: #выравнивание меток с токенами
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(label[word_idx])
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs['labels'] = labels # соединяем
    return tokenized_inputs

tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=True)


Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [10]:
seqeval = evaluate.load('seqeval') #модуль seqeval возвращает метрики для NER(pr, rec, f1) + раздельно по каждому тегу

def compute_metrics(p):
    predictions, labels = p
    predictions = predictions.argmax(axis=-1)

    true_labels = [
        [id2tag[l] for l in label if l != -100] #-100 = пропускаем игнорируемые токены (паддинги или части слова от БЕРта)
        for label in labels
    ]
    true_predictions = [
        [id2tag[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    return seqeval.compute(predictions=true_predictions, references=true_labels)


In [11]:
training_args = TrainingArguments(
    output_dir='./rubert-tiny2-ner-ne5',
    evaluation_strategy='epoch',
    save_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=20,
    weight_decay=0.01,
    #logging_dir="./logs",
    #logging_steps=10
)

#используем стандартный трейнер из берта
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['test'],
    tokenizer=tokenizer,
    data_collator=DataCollatorForTokenClassification(tokenizer),
    compute_metrics=compute_metrics
)

# Метрики до дообучения
metrics_before = trainer.evaluate()
print()
print('Метрики ДО дообучения:', metrics_before)
print()

# Обучение
trainer.train()

# Метрики после дообучения
metrics_after = trainer.evaluate()
print()
print('Метрики ПОСЛЕ дообучения:', metrics_after)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-11-d9787bc92370>:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer( #используем стандартный трейнер из берта


Trainer is attempting to log a value of "{'precision': 0.06025641025641026, 'recall': 0.34057971014492755, 'f1': 0.10239651416122005, 'number': 966}" of type <class 'dict'> for key "eval/GEOPOLIT" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.047086521483225424, 'recall': 0.08733624454148471, 'f1': 0.06118546845124282, 'number': 916}" of type <class 'dict'> for key "eval/LOC" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.01474010861132661, 'recall': 0.04337899543378995, 'f1': 0.022003474232773594, 'number': 438}" of type <class 'dict'> for key "eval/MEDIA" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.10545677498467199, 'recall': 0.


Метрики ДО дообучения: {'eval_loss': 2.3687942028045654, 'eval_model_preparation_time': 0.0012, 'eval_GEOPOLIT': {'precision': 0.06025641025641026, 'recall': 0.34057971014492755, 'f1': 0.10239651416122005, 'number': 966}, 'eval_LOC': {'precision': 0.047086521483225424, 'recall': 0.08733624454148471, 'f1': 0.06118546845124282, 'number': 916}, 'eval_MEDIA': {'precision': 0.01474010861132661, 'recall': 0.04337899543378995, 'f1': 0.022003474232773594, 'number': 438}, 'eval_ORG': {'precision': 0.10545677498467199, 'recall': 0.07603890362511052, 'f1': 0.08836372977138454, 'number': 2262}, 'eval_PER': {'precision': 0.13442211055276382, 'recall': 0.04958294717330862, 'f1': 0.07244414353419093, 'number': 4316}, 'eval_overall_precision': 0.06974552309142319, 'eval_overall_recall': 0.0914812317374691, 'eval_overall_f1': 0.07914823277748068, 'eval_overall_accuracy': 0.08406953548019379, 'eval_runtime': 1.123, 'eval_samples_per_second': 178.097, 'eval_steps_per_second': 11.576}



Epoch,Training Loss,Validation Loss,Model Preparation Time,Geopolit,Loc,Media,Org,Per,Overall Precision,Overall Recall,Overall F1,Overall Accuracy
1,No log,1.635311,0.001200,"{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 966}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 916}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 438}","{'precision': 0.16334661354581673, 'recall': 0.10875331564986737, 'f1': 0.1305732484076433, 'number': 2262}","{'precision': 0.23808678676013842, 'recall': 0.6216404077849861, 'f1': 0.3443054218800128, 'number': 4316}",0.229186,0.329175,0.270228,0.374679
2,No log,1.282965,0.001200,"{'precision': 0.8941504178272981, 'recall': 0.33229813664596275, 'f1': 0.48452830188679247, 'number': 966}","{'precision': 0.13559322033898305, 'recall': 0.034934497816593885, 'f1': 0.05555555555555556, 'number': 916}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 438}","{'precision': 0.36549607909989773, 'recall': 0.4739168877099912, 'f1': 0.4127045235803658, 'number': 2262}","{'precision': 0.3755377114998566, 'recall': 0.6068118628359592, 'f1': 0.4639503985828166, 'number': 4316}",0.385070,0.454484,0.416907,0.573027
3,No log,1.043262,0.001200,"{'precision': 0.836241610738255, 'recall': 0.644927536231884, 'f1': 0.72822910578609, 'number': 966}","{'precision': 0.5918367346938775, 'recall': 0.22161572052401746, 'f1': 0.3224781572676727, 'number': 916}","{'precision': 0.5, 'recall': 0.00228310502283105, 'f1': 0.004545454545454545, 'number': 438}","{'precision': 0.4647739221871714, 'recall': 0.5862068965517241, 'f1': 0.5184750733137831, 'number': 2262}","{'precision': 0.5279646761984861, 'recall': 0.5817886932344764, 'f1': 0.5535714285714286, 'number': 4316}",0.536154,0.524163,0.530090,0.676689
4,No log,0.873477,0.001200,"{'precision': 0.7839831401475237, 'recall': 0.7701863354037267, 'f1': 0.777023498694517, 'number': 966}","{'precision': 0.6991869918699187, 'recall': 0.37554585152838427, 'f1': 0.4886363636363636, 'number': 916}","{'precision': 0.11764705882352941, 'recall': 0.0091324200913242, 'f1': 0.01694915254237288, 'number': 438}","{'precision': 0.5392339159538863, 'recall': 0.6410256410256411, 'f1': 0.5857402544940417, 'number': 2262}","{'precision': 0.5882839991720141, 'recall': 0.6584800741427247, 'f1': 0.6214059254400349, 'number': 4316}",0.598555,0.605080,0.601800,0.728698
5,No log,0.756534,0.001200,"{'precision': 0.8157327586206896, 'recall': 0.7836438923395446, 'f1': 0.7993664202745513, 'number': 966}","{'precision': 0.7152317880794702, 'recall': 0.47161572052401746, 'f1': 0.568421052631579, 'number': 916}","{'precision': 0.14814814814814814, 'recall': 0.045662100456621, 'f1': 0.06980802792321117, 'number': 438}","{'precision': 0.606973058637084, 'recall': 0.6772767462422635, 'f1': 0.640200585039699, 'number': 2262}","{'precision': 0.6446067898581865, 'recall': 0.6950880444856349, 'f1': 0.6688963210702342, 'number': 4316}",0.649067,0.645201,0.647128,0.762753
6,No log,0.678465,0.001200,"{'precision': 0.8268817204301075, 'recall': 0.7960662525879917, 'f1': 0.8111814345991561, 'number': 966}","{'precision': 0.7383627608346709, 'recall': 0.5021834061135371, 'f1': 0.5977907732293697, 'number': 916}","{'precision': 0.25, 'recall': 0.091324200913242, 'f1': 0.13377926421404682, 'number': 438}","{'precision': 0.6203879802206163, 'recall': 0.7210433244916004, 'f1': 0.6669392762216316, 'number': 2262}","{'precision': 0.6771844660194175, 'recall': 0.7757182576459685, 'f1': 0.723110151187905, 'number': 4316}",0.672841,0.702180,0.687198,0.785266
7,No log,0.630538,0.001200,"{'precision': 0.8165983606557377, 'recall': 0.8250517598343685, 'f1': 0.8208032955715757, 'number': 966}","{'precision': 0.7392592592592593, 'recall': 0.5447598253275109, 'f1': 0.6272784412319296, 'number': 916}","{'precision': 0.410958904109589, 'recall': 0.2054794520547945, 'f1': 0.27397260273972596, 'number': 438}","{'precision': 0.6834256497656583, 'recall': 0.7091069849690539, 'f1': 0.6960295074853547, 'number': 226

/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Trainer is attempting to log a value of "{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 966}" of type <class 'dict'> for key "eval/GEOPOLIT" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 916}" of type <class 'dict'> for key "eval/LOC" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 438}" of type <class 'dict'> for key "eval/MEDIA" as a scalar. This invocation of Tensorboard's 

Trainer is attempting to log a value of "{'precision': 0.8543589743589743, 'recall': 0.8623188405797102, 'f1': 0.8583204533745492, 'number': 966}" of type <class 'dict'> for key "eval/GEOPOLIT" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.7461988304093568, 'recall': 0.6965065502183406, 'f1': 0.7204968944099378, 'number': 916}" of type <class 'dict'> for key "eval/LOC" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.6381909547738693, 'recall': 0.5799086757990868, 'f1': 0.6076555023923446, 'number': 438}" of type <class 'dict'> for key "eval/MEDIA" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.7814831066257131, 'recall': 0.787356321839


Метрики ПОСЛЕ дообучения: {'eval_loss': 0.45373550057411194, 'eval_model_preparation_time': 0.0012, 'eval_GEOPOLIT': {'precision': 0.8543589743589743, 'recall': 0.8623188405797102, 'f1': 0.8583204533745492, 'number': 966}, 'eval_LOC': {'precision': 0.7461988304093568, 'recall': 0.6965065502183406, 'f1': 0.7204968944099378, 'number': 916}, 'eval_MEDIA': {'precision': 0.6381909547738693, 'recall': 0.5799086757990868, 'f1': 0.6076555023923446, 'number': 438}, 'eval_ORG': {'precision': 0.7814831066257131, 'recall': 0.7873563218390804, 'f1': 0.7844087205461352, 'number': 2262}, 'eval_PER': {'precision': 0.7634042553191489, 'recall': 0.8313253012048193, 'f1': 0.7959183673469387, 'number': 4316}, 'eval_overall_precision': 0.770500705984577, 'eval_overall_recall': 0.7972578107439874, 'eval_overall_f1': 0.7836509251587959, 'eval_overall_accuracy': 0.853733257338273, 'eval_runtime': 0.4466, 'eval_samples_per_second': 447.785, 'eval_steps_per_second': 29.106, 'epoch': 20.0}


In [12]:
trainer.save_model('rubert-tiny2-ne5-ner')

# Задача 2: улучшение качества

## Дообучение на train-части в MLM режиме + дообучение на NER-задачу

In [13]:
from datasets import Dataset
from random import shuffle
import os
from transformers import BertForMaskedLM, DataCollatorForLanguageModeling, Trainer, TrainingArguments
from transformers import AutoTokenizer

# Собираем все train .txt файлы из папки NE5
path_to_data = '/content/Collection5'
all_files = [f for f in os.listdir(path_to_data) if f.endswith('.txt')]
shuffle(all_files)

# Разделение датасета 80/20
split_idx = int(len(all_files) * 0.8)
train_txt_files = all_files[:split_idx]
test_txt_files = all_files[split_idx:]

texts = []
for fname in train_txt_files:
    with open(os.path.join(path_to_data, fname), encoding='utf-8') as f:
        texts.append(f.read())

# Создаем HuggingFace Dataset для MLM
mlm_dataset = Dataset.from_dict({'text': texts})

# Токенизация
tokenizer = AutoTokenizer.from_pretrained('cointegrated/rubert-tiny2')

def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=128)

tokenized_dataset = mlm_dataset.map(tokenize_function, batched=True, remove_columns=['text'])

# Разделение на train и test для MLM (используем train_test_split)
dataset = tokenized_dataset.train_test_split(test_size=0.2, seed=42)

train_dataset = dataset['train']
test_dataset = dataset['test']

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

In [14]:
model = BertForMaskedLM.from_pretrained("cointegrated/rubert-tiny2")
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)

training_args = TrainingArguments(
    output_dir="./rubert-tiny2-mlm",
    overwrite_output_dir=True,
    per_device_train_batch_size=16,
    num_train_epochs=20,
    save_steps=500,
    save_total_limit=1,
    logging_steps=100,
    report_to="none",
    evaluation_strategy="no",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

# Сохраняем модель после MLM
trainer.save_model("./rubert-tiny2-mlm")

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-14-4508e2589916>:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
100,3.152400
200,2.913100
300,2.761700
400,2.684300
500,2.616800
600,2.560500
700,2.515400
800,2.494000


In [15]:
from transformers import AutoModelForTokenClassification

ner_model = AutoModelForTokenClassification.from_pretrained(
    './rubert-tiny2-mlm',
    num_labels=len(tag2id),
    id2label=id2tag,
    label2id=tag2id
)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at ./rubert-tiny2-mlm and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
from corus import load_ne5
from pathlib import Path

# Загрузка размеченных NE5 данных
records = list(load_ne5(Path('/content/Collection5')))

def record_to_dict_bio(record):
    tokens = []
    tags = []
    for span in record.spans:
        words = span.text.split()
        if not words:
            continue
        for i, word in enumerate(words):
            tokens.append(word)
            if i == 0:
                tags.append(f'B-{span.type}')
            else:
                tags.append(f'I-{span.type}')
    return {'tokens': tokens, 'ner_tags': tags}

# Преобразуем все записи в словари
data = [record_to_dict_bio(r) for r in records]

# Собираем список всех уникальных тегов
all_tags = sorted(set(tag for item in data for tag in item['ner_tags']))
tag2id = {tag: i for i, tag in enumerate(all_tags)}
id2tag = {i: tag for tag, i in tag2id.items()}

# Преобразуем теги в id
for item in data:
    item['ner_tags'] = [tag2id[tag] for tag in item['ner_tags']]

# Создаем dataset и делим
ner_dataset = Dataset.from_list(data)
ner_dataset = ner_dataset.train_test_split(test_size=0.2, seed=42)

train_dataset = ner_dataset['train']
test_dataset = ner_dataset['test']

In [17]:
# Токенизация для задачи NER
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples['tokens'],
        truncation=True,
        is_split_into_words=True
    )

    labels = []
    for i, label in enumerate(examples['ner_tags']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(label[word_idx])
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs['labels'] = labels
    return tokenized_inputs

# Токенизация данных для NER
tokenized_train = train_dataset.map(tokenize_and_align_labels, batched=True)
tokenized_test = test_dataset.map(tokenize_and_align_labels, batched=True)

# Оценка
seqeval = evaluate.load('seqeval')

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [18]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [19]:
def compute_metrics(p):
    predictions, labels = p
    predictions = predictions.argmax(axis=-1)

    true_labels = [
        [id2tag[l] for l in label if l != -100]
        for label in labels
    ]
    true_predictions = [
        [id2tag[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    return seqeval.compute(predictions=true_predictions, references=true_labels)

# Настройки для NER
training_args = TrainingArguments(
    output_dir='./rubert-tiny2-ner-mlm',
    evaluation_strategy='epoch',
    save_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=20,
    weight_decay=0.01,
)

ner_trainer = Trainer(
    model=ner_model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

# Метрики до дообучения
metrics_before = ner_trainer.evaluate()
print('Метрики ДО дообучения:', metrics_before)

# Обучение
ner_trainer.train()

# Метрики после дообучения
metrics_after = ner_trainer.evaluate()
print('Метрики ПОСЛЕ дообучения:', metrics_after)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-19-c5ddba520949>:28: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  ner_trainer = Trainer(


Trainer is attempting to log a value of "{'precision': 0.03573883161512027, 'recall': 0.05508474576271186, 'f1': 0.04335139641517299, 'number': 944}" of type <class 'dict'> for key "eval/GEOPOLIT" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.051236116087423865, 'recall': 0.275, 'f1': 0.08637873754152825, 'number': 1040}" of type <class 'dict'> for key "eval/LOC" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.015862944162436547, 'recall': 0.05555555555555555, 'f1': 0.024679170779861793, 'number': 450}" of type <class 'dict'> for key "eval/MEDIA" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.11705006765899864, 'recall': 0.072597566093

Метрики ДО дообучения: {'eval_loss': 2.3978688716888428, 'eval_model_preparation_time': 0.0009, 'eval_GEOPOLIT': {'precision': 0.03573883161512027, 'recall': 0.05508474576271186, 'f1': 0.04335139641517299, 'number': 944}, 'eval_LOC': {'precision': 0.051236116087423865, 'recall': 0.275, 'f1': 0.08637873754152825, 'number': 1040}, 'eval_MEDIA': {'precision': 0.015862944162436547, 'recall': 0.05555555555555555, 'f1': 0.024679170779861793, 'number': 450}, 'eval_ORG': {'precision': 0.11705006765899864, 'recall': 0.07259756609315988, 'f1': 0.0896140896140896, 'number': 2383}, 'eval_PER': {'precision': 0.245, 'recall': 0.0779899954524784, 'f1': 0.11831666091755776, 'number': 4398}, 'eval_overall_precision': 0.07649464798537986, 'eval_overall_recall': 0.09538795442213782, 'eval_overall_f1': 0.08490292668791655, 'eval_overall_accuracy': 0.09561835983612249, 'eval_runtime': 0.4947, 'eval_samples_per_second': 404.324, 'eval_steps_per_second': 26.281}


Epoch,Training Loss,Validation Loss,Model Preparation Time,Geopolit,Loc,Media,Org,Per,Overall Precision,Overall Recall,Overall F1,Overall Accuracy
1,No log,1.621497,0.000900,"{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 944}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 1040}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 450}","{'precision': 0.24562540505508748, 'recall': 0.1590432228283676, 'f1': 0.19307182883341822, 'number': 2383}","{'precision': 0.25152342929186805, 'recall': 0.5443383356070941, 'f1': 0.3440643863179075, 'number': 4398}",0.250701,0.300922,0.273525,0.417124
2,No log,1.260462,0.000900,"{'precision': 0.9382022471910112, 'recall': 0.3538135593220339, 'f1': 0.5138461538461538, 'number': 944}","{'precision': 0.7058823529411765, 'recall': 0.011538461538461539, 'f1': 0.02270577105014191, 'number': 1040}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 450}","{'precision': 0.364477335800185, 'recall': 0.4960134284515317, 'f1': 0.4201919658727337, 'number': 2383}","{'precision': 0.4028436018957346, 'recall': 0.5604820372896772, 'f1': 0.46876485689835506, 'number': 4398}",0.410169,0.433315,0.421425,0.583709
3,No log,1.030319,0.000900,"{'precision': 0.8748091603053435, 'recall': 0.6069915254237288, 'f1': 0.7166979362101312, 'number': 944}","{'precision': 0.7091503267973857, 'recall': 0.20865384615384616, 'f1': 0.32243684992570576, 'number': 1040}","{'precision': 0.07692307692307693, 'recall': 0.0022222222222222222, 'f1': 0.004319654427645788, 'number': 450}","{'precision': 0.46683837733419187, 'recall': 0.6084767100293748, 'f1': 0.5283293860448168, 'number': 2383}","{'precision': 0.5315747402078337, 'recall': 0.6048203728967713, 'f1': 0.5658370559455435, 'number': 4398}",0.539520,0.531850,0.535658,0.677106
4,No log,0.865763,0.000900,"{'precision': 0.8201438848920863, 'recall': 0.7245762711864406, 'f1': 0.7694038245219349, 'number': 944}","{'precision': 0.7329317269076305, 'recall': 0.35096153846153844, 'f1': 0.4746423927178153, 'number': 1040}","{'precision': 0.5166666666666667, 'recall': 0.06888888888888889, 'f1': 0.1215686274509804, 'number': 450}","{'precision': 0.5561980484279002, 'recall': 0.6458245908518674, 'f1': 0.5976699029126213, 'number': 2383}","{'precision': 0.6015273154493832, 'recall': 0.6984993178717599, 'f1': 0.646396633350868, 'number': 4398}",0.614181,0.617580,0.615876,0.733977
5,No log,0.751380,0.000900,"{'precision': 0.8240534521158129, 'recall': 0.7838983050847458, 'f1': 0.8034744842562432, 'number': 944}","{'precision': 0.6713554987212276, 'recall': 0.5048076923076923, 'f1': 0.5762897914379802, 'number': 1040}","{'precision': 0.8, 'recall': 0.1511111111111111, 'f1': 0.2542056074766355, 'number': 450}","{'precision': 0.6124567474048442, 'recall': 0.668485102811582, 'f1': 0.6392455858747994, 'number': 2383}","{'precision': 0.6638071895424836, 'recall': 0.7389722601182356, 'f1': 0.6993759414676135, 'number': 4398}",0.666811,0.670212,0.668507,0.773488
6,No log,0.669309,0.000900,"{'precision': 0.8186695278969958, 'recall': 0.8082627118644068, 'f1': 0.8134328358208955, 'number': 944}","{'precision': 0.7074401008827238, 'recall': 0.5394230769230769, 'f1': 0.6121112929623569, 'number': 1040}","{'precision': 0.7426470588235294, 'recall': 0.22444444444444445, 'f1': 0.3447098976109215, 'number': 450}","{'precision': 0.6343199700262271, 'recall': 0.7104490138480907, 'f1': 0.6702296120348377, 'number': 2383}","{'precision': 0.694578313253012, 'recall': 0.786493860845839, 'f1': 0.7376839411388356, 'number': 4398}",0.691588,0.713728,0.702483,0.794181
7,No log,0.616781,0.000900,"{'precision': 0.8589306029579067, 'recall': 0.7997881355932204, 'f1': 0.8283049917718047, 'number': 944}","{'precision': 0.7006960556844548, 'recall': 0.5807692307692308, 'f1': 0.6351209253417457, 'number': 1040}","{'precision': 0.7305699481865285, 'recall': 0.31333333333333335, 'f1': 0.43856920684292383, 'number': 450}","{'precision': 0.664179104477612, 'recall': 0.7096097356273605, 'f1': 0.68614

/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Trainer is attempting to log a value of "{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 944}" of type <class 'dict'> for key "eval/GEOPOLIT" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 1040}" of type <class 'dict'> for key "eval/LOC" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 450}" of type <class 'dict'> for key "eval/MEDIA" as a scalar. This invocation of Tensorboard's

Trainer is attempting to log a value of "{'precision': 0.865301724137931, 'recall': 0.850635593220339, 'f1': 0.8579059829059829, 'number': 944}" of type <class 'dict'> for key "eval/GEOPOLIT" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.7299412915851272, 'recall': 0.7173076923076923, 'f1': 0.7235693501454897, 'number': 1040}" of type <class 'dict'> for key "eval/LOC" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.7048780487804878, 'recall': 0.6422222222222222, 'f1': 0.6720930232558139, 'number': 450}" of type <class 'dict'> for key "eval/MEDIA" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.7674803475382705, 'recall': 0.7784305497272

Метрики ПОСЛЕ дообучения: {'eval_loss': 0.4399798512458801, 'eval_model_preparation_time': 0.0009, 'eval_GEOPOLIT': {'precision': 0.865301724137931, 'recall': 0.850635593220339, 'f1': 0.8579059829059829, 'number': 944}, 'eval_LOC': {'precision': 0.7299412915851272, 'recall': 0.7173076923076923, 'f1': 0.7235693501454897, 'number': 1040}, 'eval_MEDIA': {'precision': 0.7048780487804878, 'recall': 0.6422222222222222, 'f1': 0.6720930232558139, 'number': 450}, 'eval_ORG': {'precision': 0.7674803475382705, 'recall': 0.7784305497272346, 'f1': 0.7729166666666666, 'number': 2383}, 'eval_PER': {'precision': 0.7817685360218442, 'recall': 0.846293769895407, 'f1': 0.8127524838956218, 'number': 4398}, 'eval_overall_precision': 0.7774166491927029, 'eval_overall_recall': 0.8046663049376017, 'eval_overall_f1': 0.7908068042446542, 'eval_overall_accuracy': 0.8613985139920839, 'eval_runtime': 0.434, 'eval_samples_per_second': 460.856, 'eval_steps_per_second': 29.956, 'epoch': 20.0}


In [20]:
metrics_after = ner_trainer.evaluate()

Trainer is attempting to log a value of "{'precision': 0.865301724137931, 'recall': 0.850635593220339, 'f1': 0.8579059829059829, 'number': 944}" of type <class 'dict'> for key "eval/GEOPOLIT" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.7299412915851272, 'recall': 0.7173076923076923, 'f1': 0.7235693501454897, 'number': 1040}" of type <class 'dict'> for key "eval/LOC" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.7048780487804878, 'recall': 0.6422222222222222, 'f1': 0.6720930232558139, 'number': 450}" of type <class 'dict'> for key "eval/MEDIA" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.7674803475382705, 'recall': 0.7784305497272

In [21]:
model.save_pretrained("rubert-tiny2-mlm")
tokenizer.save_pretrained("rubert-tiny2-mlm")

('rubert-tiny2-mlm/tokenizer_config.json',
 'rubert-tiny2-mlm/special_tokens_map.json',
 'rubert-tiny2-mlm/vocab.txt',
 'rubert-tiny2-mlm/added_tokens.json',
 'rubert-tiny2-mlm/tokenizer.json')

**Вывод:**
После улучшения (задача 2) модель дала небольшой, но стабильный прирост по всем основным метрикам. Особенно по точности (precision) и F1. Это подтверждает, что дообучение на синтетических данных и MLM-преподготовка действительно дают эффект, особенно на сложных классах:

Метрика	      Задача 1	Задача 2	Разница

F1 (overall)	0.7837	0.7908	    +0.0071 ✅

Precision	    0.7705	0.7774	    +0.0069 ✅

Recall	      0.7973	0.8047	    +0.0074 ✅

Accuracy	    0.8537	0.8614	    +0.0077 ✅